In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [3]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [5]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [6]:

# from langchain_community.document_loaders import PyPDFLoader
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        # if '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版' not in filename:
        #     continue
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        pages += [loader.load()]
    # break # too much time, only load one file
pages

# loader = PyPDFLoader(os.path.join(dir, '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf'))
# pages += loader.load()
# pages = [pages[9]]
# pages

Processing documents: 100%|██████████| 32/32 [00:36<00:00,  1.14s/it]


[[Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='資安等級：內部\n台灣人壽\n個人保險首/續期繳費暨保單貸款還款息作業手冊\n(113 年 7 月版)_業務通路版'),
  Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 2}, page_content='\n\n1. 序號：1\n   主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明。\n   備註：-\n\n### 繳費管道變更與繳費日認定原則\n\n#### 原指定繳費管道\n\n1. **金融機構**\n   - **變更為自行匯款**\n     - 若於一照期限內完成匯款，可以用原授權書之申請日為繳費日。\n     - 若未於一照期限內完成匯款，以實際匯款日為繳費日。\n   - **變更為他行金融機構轉帳**\n     - 不限，可以用原授權書之申請日為繳費日。\n   - **變更為信用卡**\n     - 不限，以新信用卡授權書之申請日為繳費日。\n\n2. **信用卡**\n   - **變更為自行匯款**\n     - 不限，以實際匯款日為繳費日。\n   - **變更為金融機構轉帳**\n     - 不限，以新金融機構轉帳授權書之申請日為繳費日。\n   - **變更為同銀行其他信用卡或他行信用卡**\n     - 不限，可以用原信用卡授權書之申請日為繳費日。\n\n#### 註釋\n\n- **註1**：一照意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者。\n- **註2**：停售商品變更繳費管道後之繳費日若等於或晚於商品停售日，應予以退件。\n\n### 繳費方式\n\n- **金融機構轉帳**\n- **信用卡繳費**\n- **自行匯款**\n- **支票**\n\n### 應檢附文件\n\n- **金融機構轉帳**：保險費暨保險單借款利息自動轉帳付款授權書\n- **信用卡繳費**：保險費信用

In [7]:
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=5)
chunk_list = graph_builder.build_chunk_graph_with_parent_child(pages)
chunk_list

[{'chunk_id': '5ed34c2c-fbd2-4c0b-be82-2023d051094a',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'total_page_num': 28}, page_content='資安等級：內部 台灣人壽 個人保險首/續期繳費暨保單貸款還款息作業手冊 (113 年 7 月版)_業務通路版   1. 序號：1    主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明。    備註：-  ### 繳費管道變更與繳費日認定原則  #### 原指定繳費管道  1. **金融機構**    - **變更為自行匯款**      - 若於一照期限內完成匯款，可以用原授權書之申請日為繳費日。      - 若未於一照期限內完成匯款，以實際匯款日為繳費日。    - **變更為他行金融機構轉帳**      - 不限，可以用原授權書之申請日為繳費日')},
 {'chunk_id': '8cc4b208-e454-420b-97e8-43857c445f6a',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'total_page_num': 28}, page_content='。    - **變更為信用卡**      - 不限，以新信用卡授權書之申請日為繳費日。  2. **信用卡**    - **變更為自行匯款**      - 不限，以實際匯款日為繳費日。    - **變更為金融機構轉帳**      - 不限，以新金融機構轉帳授權書之申請日為繳費日。    - **變更為同銀行其他信用卡或他行信用卡**      - 不限，可以用原信用卡授權書之申請日為繳費日。  #### 註釋  - **註1**：一照意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者')},
 {'chunk_id': '69b38e5d-bf15-4912-99ce-57fc1a152850',
  'chu

In [ ]:
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for chunk in chunk_list:
#     doc = chunk['chunk_doc']
#     content = doc.page_content
#     curr_num = num_tokens_from_string(content)
#     tokens_num += num_tokens_from_string(content)
# tokens_num

In [8]:
graph_builder._get_combined_chunks(chunk_list)

[Document(metadata={'combined_chunk_ids': ['c475b78a-9b67-4d6a-a70c-5e8bce455662']}, page_content='資安等級：內部 台灣人壽 個人保險首/續期繳費暨保單貸款還款息作業手冊 (113 年 7 月版)_業務通路版   1. 序號：1    主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明'),
 Document(metadata={'combined_chunk_ids': ['8b6e4073-8706-42c4-9955-5efb1135d3c2']}, page_content='。    備註：無  ### 繳費管道變更規則  #### 原指定繳費管道：金融機構  1. **改自行匯款**    - **於一照期限內完成**：是，於一照期限內完成匯款    - **繳費日認定原則**：可以用原授權書之申請日為繳費日    - **未於一照期限內完成**：否，未於一照期限內完成匯款    - **繳費日認定原則**：以實際匯款日為繳費日  2. **改他行金融機構轉帳**    - **完成**：不限    - **繳費日認定原則**：可以用原授權書之申請日為繳費日  3. **改信用卡**    - **完成**：不限    - **繳費日認定原則**：以新信用卡授權書之申請日為繳費日  #### 原指定繳費管道：信用卡  1. **改自行匯款**    - **完成**：不限    - **繳費日認定原則**：以實際匯款日為繳費日  2. **改金融機構轉帳**    - **完成**：不限    - **繳費日認定原則**：以新金融機構轉帳授權書之申請日為繳費日  3. **改同銀行其他信用卡或改他行信用卡**    - **完成**：不限    - **繳費日認定原則**：可以用原信用卡授權書之申請日為繳費日  ### 註釋  - **註1**：一照：意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者'),
 Document(metadata={'combined_chunk_ids': ['79faa07a-6722-49e3-9f4e-26853ec9d311']}, page_content='。 

In [8]:

graph_documents = graph_builder.get_graph_from_llm(llm, chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

100%|██████████| 99/99 [02:15<00:00,  1.37s/it]


[GraphDocument(nodes=[Node(id='繳款聲明書', type='Document', properties={'description': 'X'}), Node(id='文件', type='Document', properties={'description': '可確認身分/關係為要保人、被保險人、受益人之一者，無須檢附'})], relationships=[], source=Document(metadata={'combined_chunk_ids': ['6f549e0e-011d-4cb3-9fc0-c6025b4aee2a']}, page_content='【註】 - **繳款聲明書**：X  【註】如要保文件可確認身分/關係為要保人、被保險人、受益人之一者，無須檢附')),
 GraphDocument(nodes=[Node(id='停售商品', type='Concept', properties={'description': '商品不再銷售'}), Node(id='繳費管道', type='Concept', properties={'description': '繳費的方式或途徑'}), Node(id='繳費日', type='Concept', properties={'description': '繳費的日期'}), Node(id='商品停售日', type='Concept', properties={'description': '商品停止銷售的日期'})], relationships=[Relationship(source=Node(id='繳費日', type='Concept', properties={}), target=Node(id='商品停售日', type='Concept', properties={}), type='等於或晚於', properties={}), Relationship(source=Node(id='停售商品', type='Concept', properties={}), target=Node(id='繳費管道', type='Concept', properties={}), type='變更', properties={}), Rel

In [9]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [10]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='繳款聲明書', type='Document', properties={'description': 'X'}), Node(id='文件', type='Document', properties={'description': '可確認身分/關係為要保人、被保險人、受益人之一者，無須檢附'})], relationships=[], source=Document(metadata={'combined_chunk_ids': ['6f549e0e-011d-4cb3-9fc0-c6025b4aee2a'], 'id': '3aec87baf13308cd2ef85f5cdd9c971a'}, page_content='【註】 - **繳款聲明書**：X  【註】如要保文件可確認身分/關係為要保人、被保險人、受益人之一者，無須檢附')),
 'chunk_id': '6f549e0e-011d-4cb3-9fc0-c6025b4aee2a'}

In [11]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)